In [1]:
import torch
import torch.nn as nn

### 下面验证 transpose 和 permute 的用法

In [2]:
x= torch.randn(12).reshape(3,2,2)
x

tensor([[[-1.3279, -1.0326],
         [ 1.2334, -1.0557]],

        [[ 0.3534, -0.5765],
         [ 0.0330,  0.3341]],

        [[-1.3845,  0.4735],
         [ 0.6970,  0.5615]]])

In [3]:
x_t = x.transpose(2,1)
x_t

tensor([[[-1.3279,  1.2334],
         [-1.0326, -1.0557]],

        [[ 0.3534,  0.0330],
         [-0.5765,  0.3341]],

        [[-1.3845,  0.6970],
         [ 0.4735,  0.5615]]])

In [4]:
x_t = x.permute(0,2,1)
x_t

tensor([[[-1.3279,  1.2334],
         [-1.0326, -1.0557]],

        [[ 0.3534,  0.0330],
         [-0.5765,  0.3341]],

        [[-1.3845,  0.6970],
         [ 0.4735,  0.5615]]])

### 多头注意力的实验

In [5]:
batch_size = 3
dim_embedding = 16
num_heads = 8
dim_qk = 16
dim_v =16
sequence_length = 4

In [6]:
x = torch.randn(batch_size,sequence_length,dim_embedding)
x.shape

torch.Size([3, 4, 16])

In [7]:
linear_q = nn.Linear(dim_embedding,dim_qk, bias=False)
linear_k = nn.Linear(dim_embedding,dim_qk, bias=False)
linear_v = nn.Linear(dim_embedding,dim_v, bias = False)

normal_factor = ( dim_qk // num_heads) ** -0.5 

In [8]:
div_q = dim_qk // num_heads
div_v = dim_v // num_heads
div_q, div_v

(2, 2)

In [9]:
q= linear_q(x).view(batch_size,sequence_length,num_heads, div_q).permute(0,2,1,3)

q.shape

torch.Size([3, 8, 4, 2])

In [10]:
k= linear_k(x).view(batch_size,sequence_length,num_heads, div_q).permute(0,2,1,3)
k.shape

torch.Size([3, 8, 4, 2])

In [11]:
v= linear_v(x).view(batch_size,sequence_length,num_heads,div_v).permute(0,2,1,3)
v.shape

torch.Size([3, 8, 4, 2])

In [12]:
dist = torch.matmul(q,k.permute(0,1,3,2)) ** normal_factor
dist.shape

torch.Size([3, 8, 4, 4])

In [13]:
atten_socre  = torch.matmul(dist, v) 
atten_socre.shape

torch.Size([3, 8, 4, 2])

In [14]:
atten_socre = atten_socre.permute(0,2,1,3)

atten_socre.shape

torch.Size([3, 4, 8, 2])

In [15]:
atten_socre = atten_socre.contiguous().view(batch_size,sequence_length,dim_v)
atten_socre.shape

torch.Size([3, 4, 16])

In [23]:
linear = nn.Linear(2,2)
linear2 = nn.Linear(2,4)
model =  nn.Sequential(linear,linear2)

In [36]:
for param in model.parameters():
    print(param)
for module in list(model.modules()):
    for key,val in module._modules.items():
        print(key,val)

Parameter containing:
tensor([[ 0.0530, -0.0046],
        [-0.5766, -0.7019]], requires_grad=True)
Parameter containing:
tensor([ 0.3891, -0.0369], requires_grad=True)
Parameter containing:
tensor([[ 0.4832, -0.2867],
        [ 0.5267,  0.6571],
        [ 0.3882,  0.5172],
        [ 0.1580,  0.4386]], requires_grad=True)
Parameter containing:
tensor([-0.5923,  0.0330,  0.5836,  0.6242], requires_grad=True)
0 Linear(in_features=2, out_features=2, bias=True)
1 Linear(in_features=2, out_features=4, bias=True)


In [39]:
class Person():
    def __init__(self, name, age):
        self.name = name
        self.age = age

p = Person('John', 36)

setattr(p, 'name', 'John')
p.name

'John'

In [42]:
setattr(p, 'sex', 'female')

In [43]:
p.sex,p.age,p.name

('female', 36, 'John')

In [50]:
lst = [[1,2,3]]
lst*3
## 等价与 [item*3 for item in lst]

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [51]:
for (key1,value1),(key2,value2) in [(1,2),(3,4)]:
    print(key1,value1)
    print("dfsa")
    print(key2,value2)

TypeError: cannot unpack non-iterable int object

In [1]:
import torch

In [2]:
mask = torch.triu(,diagonal=1)

In [3]:
mask

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])

In [6]:
k = torch.ones(3,3)

In [10]:
k,k.shape

(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 torch.Size([3, 3]))

In [13]:
v = torch.full((3,3),float(2))

In [14]:
v,v.shape

(tensor([[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]]),
 torch.Size([3, 3]))

In [20]:
attention_socre = torch.matmul(k,v)

In [21]:
attention_socre

tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]])

In [22]:
mask = torch.full((3,3),float("-inf"))
mask = torch.triu(mask,diagonal=1)
attention_socre = attention_socre + mask
attention_socre 

tensor([[6., -inf, -inf],
        [6., 6., -inf],
        [6., 6., 6.]])

In [38]:
def time_cal(func):
    import time
    start = time.time()
    func()
    end = time.time()
    print("time cost is ", end-start)
    
    
def add(i,j):
    return i+j

time_cal(lambda: add(1,2))

TypeError: 'int' object is not callable

In [44]:
from pprint import pprint
k = torch.ones((3,3))
v = torch.full((3,3),float(2))


import time 
start = time.time()
for _ in range(1000):
    torch.matmul(k,v)
end = time.time()
print(end-start)

def two_for_test():
    for _ in range(1000):
        res = []
        for i,k_i in enumerate(k):
            line = []
            for j,v_i in enumerate(v):
                line.append(torch.matmul(k_i,v_i))
                if i== j: break
            res.append(line)
def one_for_test():
    for _ in range(1000):
        res =  []
        for i,k_i in enumerate(k) :
            line = torch.matmul(k_i,v[:,:i+1])     
            res.append(line)

time_cal(two_for_test) 
time_cal(one_for_test)

0.0024263858795166016
time cost is  0.025965452194213867
time cost is  0.022052288055419922


In [45]:
v_i = torch.tensor([[1,2,1]])

torch.cat([v,v_i],dim=0)

tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.],
        [1., 2., 1.]])